<a href="https://colab.research.google.com/github/mavemouth/JAGBot-Public/blob/main/JAGBot_Public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**STEP 1 :**

**Install the following packages:-**
*   langchain : community , groq
*   Sentence transformers
*   ChromaDB
*   PyPDF2
*   gradio
*   pypdf









In [ ]:
!pip install langchain langchain-community langchain-groq sentence-transformers chromadb PyPDF2 gradio pypdf

**STEP-2 :-**

**Importing the Libraries :-**
*  **PyPDFloader** :- Loads PDF documents.
*  **RecursiveCharacterTextSplitter** :- Splits documents into smaller chunks.
*  **HuggingFaceEmbeddings** :- Creates numerical represntation of text.
*  **Chroma** :- Vector database to store and search document chunks.
*  **RetrivalQA** :- A system that retrieves relevant info and generates answers.
*  **ChatGroq** :- An LLM.
*  **Gradio** :- For creating the web interface.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
import gradio as gr

**STEP-3 :- Loading PDF's**

* **loader1** and **loader2** contains the pdf's we require.
* **documents** is the combination of these two documents.

In [ ]:
loader1 = PyPDFLoader("armyrules1954.pdf")
loader2 = PyPDFLoader("armyact1950.pdf")
documents = loader1.load() + loader2.load()

**STEP-4 :- Chunking documents**

* Splits documents into smaller pieces (1000 characters each).

* Overlap of 200 characters helps maintain context.



In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(documents)

**STEP-5 :- Embedding and VectorDB**

* **Embeddings**: Convert text to numbers (vectors) that capture meaning.

* **Chroma**: Stores these vectors for fast searching.

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(chunks, embeddings)

/tmp/ipython-input-5-1181982184.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models o

**STEP-6 :- Groq LLM model**

In this cell, we are generally calling the **Groq** model thorough its API key.

In [ ]:
llm = ChatGroq(
    temperature=0.3,
    groq_api_key = "Place your API key here.",
    model_name="llama3-70b-8192"
)


**STEP-7 :- Retrieval-based QA**

Combines the retriever (finds relevant chunks) and LLM (generates answers).

When you ask a question:

  * Finds the most relevant document chunks

  * Passes them to the LLM to generate an answer

In [ ]:
# Retrieval-based QA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=False
)


**STEP-8 :- Gradio UI**

Creates a simple web interface using Gradio.

In [ ]:
def respond(message, history):
    result = qa_chain.run(message)
    return result

**STEP-9 :- Launch Gradio Chatbot**

In [ ]:
gr.ChatInterface(fn=respond, title="🪖 JAGBot – Military Law Assistant").launch()

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fb7060d3c87fdab3c9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
